<h2 style="text-align: center; font-weight: bold">Genetic Algorithm Crossovers for Travelling Salesperson Problem</h2>

In [63]:
import random
from IPython.core.display import Image

#### Fixed example, 15 cities 

In [64]:
Px = ['O', 'D', 'G', 'L', 'A', 'H', 'K', 'M', 'B', 'J', 'F', 'C', 'N', 'I', 'E']
Py = ['H', 'G', 'M', 'F', 'O', 'A', 'D', 'K', 'I', 'C', 'N', 'E', 'L', 'B', 'J']

#### Randomly generated example, 26 cities

In [65]:
# P_n - parent_, C_n - children
P1 = [chr(i) for i in range(65, 91)]
P2 = [chr(i) for i in range(65, 91)]
random.shuffle(P1) 
random.shuffle(P2)

#### Printing Function - utility

In [66]:
def tsp_printer(cities: list[str], list_name: str, strong_genes_seq: tuple[int, int], color_code: int) -> None:
  print(list_name, end = ": ")
  for i in range(len(cities)):
    if i not in range(strong_genes_seq[0], strong_genes_seq[1]):
      print(cities[i], end = " ")
    else:
      print(f"\033[38;5;{color_code}m{cities[i]}", end = " \033[0m")
  print()

In [67]:
tsp_printer(P1, "P1", (9, 16), 208)
tsp_printer(P2, "P2", (9, 16), 208)

tsp_printer(Px, "Px", (5, 10), 208)
tsp_printer(Py, "Py", (5, 10), 208)

P1: X R T S E N V M D B J O P A G C U K Y Q F W I Z L H 
P2: P S Y O F J B T E N Z W U G K L M A I R Q X V D C H 
Px: O D G L A H K M B J F C N I E 
Py: H G M F O A D K I C N E L B J 


#### 1. Ordered Crossover

In [ ]:
Image(filename="../Graphs/")

In [68]:
def ordered_crossover(P1: list[str], P2: list[str], strong_genes_seq: tuple[int, int]) -> tuple[list[str], list[str]]:  
  """
  "strong_genes_seq" should have the start and end indices of strong genes { [start_index, end_index) },\n 
  i.e, the range of genes that are copied untouched in corresponding children (P_n Ξ C_n)
  """
  C1, C2 = [], []
  for i in range(strong_genes_seq[0], strong_genes_seq[1]):
    C1.append(P1[i])
    C2.append(P2[i])

  weak_genes_C1 = list(filter(lambda x: x if x not in C1 else None, P2))   # 'x' is Truthy when parameter is 'x' whereas 'None' is falsy when param is 'x'... that's why not using explicit boolean returns is still working
  weak_genes_C2 = list(filter(lambda x: x if x not in C2 else None, P1))
  
  # As I've selected a middle slice to be the strong genes and C1, C2 originally only having those at the beginning, re-arrangement is required, kinda like an oversimplified CRISPR for retaining exact parent features heh
  C1 = weak_genes_C1[:strong_genes_seq[0]] + C1 + weak_genes_C1[strong_genes_seq[0]:]
  C2 = weak_genes_C2[:strong_genes_seq[0]] + C2 + weak_genes_C2[strong_genes_seq[0]:]

  return C1, C2

# not my first time using the filter function but this is a good read: https://realpython.com/python-filter-function/

##### Example 1 - Random 26 Cities

In [69]:
C1, C2 = ordered_crossover(P1, P2, (9, 16))
print("Parents were: ")
tsp_printer(P1, "P1", (9, 16), 208)
tsp_printer(P2, "P2", (9, 16), 208)

print("\nChildren are: ")
tsp_printer(C1, "C1", (9, 16), 51)
tsp_printer(C2, "C2", (9, 16), 51)

Parents were: 
P1: X R T S E N V M D B J O P A G C U K Y Q F W I Z L H 
P2: P S Y O F J B T E N Z W U G K L M A I R Q X V D C H 

Children are: 
C1: S Y F T E N Z W U B J O P A G C K L M I R Q X V D H 
C2: X R T S E V M D B N Z W U G K L J O P A C Y Q F I H 


##### Example 2 - Fixed 15 cities (manually solved, performed for validation)

In [70]:
Cx, Cy = ordered_crossover(Px, Py, (5, 10))
print("Parents were: ")
tsp_printer(Px, "Px", (5, 10), 208)
tsp_printer(Py, "Py", (5, 10), 208)

print("\nChildren are: ")
tsp_printer(Cx, "Cx", (5, 10), 51)
tsp_printer(Cy, "Cy", (5, 10), 51)

Parents were: 
Px: O D G L A H K M B J F C N I E 
Py: H G M F O A D K I C N E L B J 

Children are: 
Cx: G F O A D H K M B J I C N E L 
Cy: O G L H M A D K I C B J F N E 


#### 2. Partially Mapped Crossover

In [71]:
def partially_mapped_crossover(P1: list[str], P2: list[str], SGS_endices: tuple[int, int]) -> tuple[list[str], list[str]]:
  """
  "SGS_endices => Strong Genes Sequence ends-indices" should have the start and end indices of strong genes { [start_index, end_index) },\n 
  i.e, the range of genes that are copied untouched in corresponding children (P_n Ξ C_n)
  """
  SGs_C1, SGs_C2 = P1[SGS_endices[0]: SGS_endices[1]], P2[SGS_endices[0]: SGS_endices[1]]   # the genes that get copied in corr. children as it is

  # Done so to copy most of the weak genes as it is (the ones not mapped to anything) as well as leaving out space for the mapped ones in corresponding children
  C1 = [city if city not in SGs_C1 else chr(33) for city in P2] 
  C2 = [city if city not in SGs_C2 else chr(33) for city in P1]
  
  # strong genes pass down as it is. for the 15 cities example, A D K I C got replaced by H K M B J so now we need to find new places in C1 for A D I C (not K since it's already placed)
  for i in range(SGS_endices[0], SGS_endices[1]):
    C1[i] = P1[i]
    C2[i] = P2[i]
  
  for i in range(SGS_endices[0], SGS_endices[1]):
    if P2[i] not in SGs_C1:
      to_ix = P2.index(P1[i])         # using Parents since i ranges in endices, if it would've been from 0 then SGs_n lists could've been used
      while to_ix in range(SGS_endices[0], SGS_endices[1]):
        to_ix = P2.index(P1[to_ix])
      C1[to_ix] = P2[i]
  
  for i in range(SGS_endices[0], SGS_endices[1]):
    if P1[i] not in SGs_C2:
      to_ix = P1.index(P2[i])         # char in P1's SGs p_mapped to char in P2 SGs
      while to_ix in range(SGS_endices[0], SGS_endices[1]):
        to_ix = P1.index(P2[to_ix])
      C2[to_ix] = P1[i]

  return C1, C2

##### Example 1 - Fixed 15 cities (manually solved, performed for validation)

In [72]:
Cx, Cy = partially_mapped_crossover(Px, Py, (5, 10))
print("Parents were: ")
tsp_printer(Px, "Px", (5, 10), 208)
tsp_printer(Py, "Py", (5, 10), 208)

print("\nChildren are: ")
tsp_printer(Cx, "Cx", (5, 10), 51)
tsp_printer(Cy, "Cy", (5, 10), 51)

Parents were: 
Px: O D G L A H K M B J F C N I E 
Py: H G M F O A D K I C N E L B J 

Children are: 
Cx: A G D F O H K M B J N E L I C 
Cy: O M G L H A D K I C F J N B E 


##### Example 2 - Random 26 Cities

In [73]:
C1, C2 = partially_mapped_crossover(P1, P2, (9, 16))
print("Parents were: ")
tsp_printer(P1, "P1", (9, 16), 208)
tsp_printer(P2, "P2", (9, 16), 208)

print("\nChildren are: ")
tsp_printer(C1, "C1", (9, 16), 51)
tsp_printer(C2, "C2", (9, 16), 51)

Parents were: 
P1: X R T S E N V M D B J O P A G C U K Y Q F W I Z L H 
P2: P S Y O F J B T E N Z W U G K L M A I R Q X V D C H 

Children are: 
C1: U S Y W F Z N T E B J O P A G C M K I R Q X V D L H 
C2: X R T S E B V M D N Z W U G K L P A Y Q F O I J C H 
